In [1]:
import numpy as np
import pylas
from scipy.spatial import KDTree
from sklearn.utils import resample

# 1. Load the data

In [2]:
# load the data and save as numpy array
f = pylas.read('your_file.las')
f2 = pylas.read('your_file.las')
coord_tr1 = np.c_[f.x, f.y, f.z, f.red, f.green, f.blue]
coord_tr2 = np.c_[f2.x, f2.y, f2.z, f2.red, f2.green, f2.blue, f2.label_ch]

# 2. Order the data so the coordinates match

In [3]:
# Assume you have two point clouds, 'cloud1' and 'cloud2', with different densities
cloud1 = coord_tr1
cloud2 = coord_tr2

# Define the desired number of points for the resampled dataset
n_points = int(len(coord_tr1))

# Resample the combined point cloud to the desired number of points
cloud1_resampled = resample(cloud1, n_samples=n_points, random_state=42)

# Split the resampled point cloud back into two separate point clouds
cloud2_resampled = resample(cloud2, n_samples=n_points, random_state=42)

In [4]:
# Construct KDTree from array2
tree = KDTree(cloud2_resampled[:, :2])

# Find the closest point in array2 to each point in array1
dist, ind = tree.query(cloud1[:, :2])

# Reorder array2 based on ind
array2_reordered = cloud2_resampled[ind]

# 3. Prepare features for the algorithm

In [5]:
# Filter only the features we will use for prediction
before_features1 = coord_tr1[:, 2:6]
before_features2 = array2_reordered[:, 2:6]
targets = array2_reordered[:, 6]

In [6]:
# Combine features in one array
result_substraction = before_features1 - before_features2
result_addition = before_features1 + before_features2

In [8]:
train_data = np.c_[result_addition, result_substraction]

# 4. Stack the data and save

In [ ]:
# train_data_stack = np.stack((a, b))
# targets_stack = np.stack((a, b))

In [ ]:
np.save('train_data.npy', train_data)
np.save('targets.npy', targets)